In [ ]:
# This line is a convenience to import most packages you'll need. You may need to import others (eg random and cmath)
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math
from IPython.display import Audio

%matplotlib inline

from scipy.fftpack import fft
from scipy.signal import hann
from __future__ import division

In [ ]:
from scipy.fftpack import fft
from scipy.signal import hann

def stft(signal, window_size, hop_size, window_type = 'hann'):
    """
    Computes the short term fourier transform of a 1-D numpy array, where the array 
    is windowed into a set of subarrays, each of length window_size. The distance between
    window centers (in samples) is given by hop_size. The type of window applied is
    determined by window_type. This returns a 2-D numpy array where the ith column
    is the FFT of the ith window. Each column contains an array of complex values.
    
    Input Parameters
    ----------------
    signal: The 1-d (complex or real) numpy array containing the signal
    window_size: an integer scalar specifying the number of samples in a window
    hop_size: an integer specifying the number of samples between the start of adjacent windows
    window_type: a string specifying one of two "hann" or "rectangular"
    
    Returns
    -------
    a 2D numpy array of complex numbers where the array column is the FFT of the ith window,
    and the jth element in the ith column is the jth frequency of analysis.
    """
    
    # figure out how many hops
    length_to_cover_with_hops = len(signal) - window_size;
    assert (length_to_cover_with_hops >= 0), "window_size cannot be longer than the signal to be windowed"
    num_hops = int(1 + length_to_cover_with_hops/hop_size);
    
    # make our window function
    if (window_type == 'hann'):
        window = sp.signal.hann(window_size, sym=False)
    else:
        window = np.ones(window_size)
    
    stft = [0]*num_hops
    # fill the array with values 
    for hop in range(num_hops):
        start = hop*hop_size
        end = start + window_size
        unwindowed_sound = signal[start:end]
        windowed_sound =  unwindowed_sound * window
        stft[hop]= fft(windowed_sound, window_size) 
    return np.array(stft).T

In [ ]:
from scipy.fftpack import ifft

def istft(X, hop_size):
    """
    Takes a 2-D numpy array representing an STFT of some signal, where stft[i] 
    is the FFT of the ith window as input and stft[i,k] is the kth frequency of analysis.
    Performs an inverse FFT on each window and then does overlap & add resynthesis to rebuild 
    the original signal the STFT was built from.
    
    Input Parameters
    ----------------
    X: a 2-D numpy array of complex numbers representing an STFT, where the ith 
    column is the FFT of the ith window, and the jth row is the jth frequency of analysis.
        
    hop_size: an integer specifying the number of samples between the start of adjacent windows.
        
    Returns
    -------
    a 1-d numpy array of (possibly complex) values representing the original signal used to make X
    """
    
    # make an empty signal of the appropriate length
    window_size,num_hops = X.shape
    signal_length = (num_hops-1)*hop_size + window_size 
    signal = np.zeros(signal_length,dtype='complex');
    
    #fill the signal
    for n in range(num_hops):
        start = n * hop_size
        end = start + window_size
        signal[start:end] = signal[start:end] + ifft(X[:,n])
    return signal

In [ ]:
def plt_spectrogram(X,win_length, hop_size, sample_rate, zoom_x=None, zoom_y=None,tick_labels='time-freq'):
    """
    Plots the log magnitude spectrogram.
    
    Input Parameters:
    ------------------
    X: 2D complex numpy array containing the stft values. Rows correspond to frequency bins and columns to time frames.
    win_length: the length of the analysis window
    hop_size: the hop size between adjacent windows
    sample_rate: sampling frequency
    tick_labels: the type of x and y tick labels, there are two options:
                 'time-freq': shows times (sec) on the x-axis and frequency (Hz) on the y-axis (default)
                 'bin-frame': shows time frame number on the x-axis and frequency bin number on the y-axis
                
    zoom_x: 1 by 2 numpy array containing the range of values on the x-axis, e.g. zoom_t=np.array([x_start,x_end])
    zoom_y: 1 by 2 numpy array containing the range of values on the y-axis, e.g. zoom_f=np.array([y_start,y_end])
    
    
    Returns:
    ---------
    times: 1D real numpy array containing time instances corresponding to stft frames
    freqs: 1D real numpy array containing frequencies of analyasis up to Nyquist rate
    2D plot of the magnitude spectrogram
    """
    
    # Find the size of stft
    Nf,Nt=np.shape(X)
    
    # Compute the log magnitude spectrogram
    X=20*np.log10(np.abs(X))
    
    # Extract the first half of the spectrum for each time frame
    X=X[0:Nf/2]
    Nf=np.shape(X)[0]
        
    # Generate time vector for plotting
    times=(hop_size/float(sample_rate))*np.arange(Nt)
    
    # Generate frequency vector for plotting
    freqs=(float(sample_rate)/win_length)*np.arange(Nf)
    
    # Generate time and frequency matrices for pcolormesh
    times_matrix,freqs_matrix=np.meshgrid(times,freqs)
    
    # Plot the log magnitude spectrogram
    plt.title('Log magnitude spectrogram')
    if tick_labels == 'bin-frame':
        plt.pcolormesh(X)
        plt.xlabel('Time-frame Number')
        plt.ylabel('Frequency-bin Number')
    else:
        plt.pcolormesh(times_matrix,freqs_matrix,X)
        plt.xlabel('Time (sec)')
        plt.ylabel('Frequency (Hz)')

        
    # Zoom in on the plot if specified
    if zoom_x is None and zoom_y is None:
        plt.axis('tight')
        
    if zoom_x is not None:
        plt.xlim(zoom_x)
        
    if zoom_y is not None:
        plt.ylim(zoom_y)
        
    return    

In [ ]:
def HPF(X, start=None, end=None, proportion=.1):
    """
    Takes a 1-D array audio signal X as input and applies a high-pass filter to the signal
    during the time period between the index start and end. Function completes an STFT on
    the signal, then multiplies frequencies 20 Hz through 400 Hz by the proportion entered.
    
    Input Parameters
    ----------------
    X: a 2-D numpy array of real numbers, where the ith value is the ith sample in the audio signal
    
    start: indicates the index that the fade will begin on. If left blank, index number 0 will
    be chosen
    
    end: indicates the index that the fade will end on. If left blank, the last sample of the
    signal will be chosen.
    
    proportion: determines the strength of the frequencies during the filter. .5 is good to start with,
    going lower will increase the strength of the filter and get rid of more low frequencies
    
    Returns
    -------
    a 1-D numpy array of real values representing the faded output signal
    """
    
    sig_length = len(X)
    Y = np.zeros(len(X))
    
    print sig_length
    
    if (start==None):
        start = 0
    if (start < 0):
        start = 0
      
    if (end==None):
        end = sig_length - 1
    if (start < 0):
        end = sig_length - 1
        
    print start
    print end 
    
    length_to_cover_with_hops = end - start - 2048;
    num_hops = int(1 + length_to_cover_with_hops/1024);
    
    end = 2048 + (num_hops - 1)*1024 - 1 + start

    print start
    print end
    
    filter_length = 1 + end - start
    
    filtered_segment = X[start:(end+1)]
    before_segment = X[0:start]
    after_segment = X[(end+1):sig_length]
    
    window_size=2048
    hop_size=1024

    print len(filtered_segment)

    filtered_stft = stft(filtered_segment, window_size, hop_size, window_type = 'hann')
    filtered_stft[0:20,:] = filtered_stft[0:20,:] * proportion
    filtered_stft[2030:2049,:] = filtered_stft[2030:2049,:] * proportion
    
    filtered_istft = istft(filtered_stft, hop_size)

    Y[0:start] = before_segment
    Y[start:(end+1)] = filtered_istft
    Y[(end+1):sig_length] = after_segment
    
    return Y

In [ ]:
test1,sr = librosa.load('track1test.wav',44100)
filter_test = HPF(test1, proportion=.5, start=100000, end = 500000)
plt.plot(filter_test)
Audio(filter_test,rate=44100)